In [1]:
#GDD calculation for corn from class CornGDD. 
import pandas as pd

def fahrenheit_to_celsius(temp_f):
    return (temp_f - 32) * 5 / 9

def celsius_to_fahrenheit(temp_c):
    return (temp_c * 9 / 5) + 32

class CornGDD:
    def __init__(self, daily_temp_df, time_column, min_temp_column, max_temp_column, celsius=True):
        self.daily_temp_df = daily_temp_df
        self.time_column = time_column
        self.min_temp_column = min_temp_column
        self.max_temp_column = max_temp_column
        self.celsius = celsius

        # Ensure the time column is of datetime type
        self.daily_temp_df[self.time_column] = pd.to_datetime(self.daily_temp_df[self.time_column])

        # If initial data is in Celsius, optionally convert it to Fahrenheit for uniform GDD calculation
        if self.celsius:
            self.daily_temp_df[self.min_temp_column] = self.daily_temp_df[self.min_temp_column].apply(celsius_to_fahrenheit)
            self.daily_temp_df[self.max_temp_column] = self.daily_temp_df[self.max_temp_column].apply(celsius_to_fahrenheit)
            self.celsius = False  # Now all data is in Fahrenheit

    def con_gdd(self, start_date, end_date):
        start_date = pd.Timestamp(start_date)
        end_date = pd.Timestamp(end_date)

        date_mask = (self.daily_temp_df[self.time_column] >= start_date) & (self.daily_temp_df[self.time_column] <= end_date)
        masked_data = self.daily_temp_df.loc[date_mask].copy()

        gdd_values = []
        cumulative_gdd = 0

        for index, row in masked_data.iterrows():
            min_temp = max(row[self.min_temp_column], 50)  # Fahrenheit base
            max_temp = min(row[self.max_temp_column], 86)  # Fahrenheit upper limit

            mean_temp = (min_temp + max_temp) / 2
            base_temp = 50  # Fahrenheit base temperature for GDD calculation
            gdd = max(mean_temp - base_temp, 0)

            gdd_values.append(gdd)
            cumulative_gdd += gdd

        masked_data['GDD'] = gdd_values
        masked_data['Cumulative_GDD'] = masked_data['GDD'].cumsum()

        return masked_data

# This modified class converts temperatures to Fahrenheit if they start in Celsius, then computes and returns GDD in Fahrenheit.


Import min/max temp for SaintPaul from Gridded GEMS exchange

In [2]:
import pandas as pd
StPaul_GEMS = pd.read_csv("st_paul_gems_2023_2024.csv", on_bad_lines='skip')

In [3]:
StPaul_GEMS_2023 = CornGDD(StPaul_GEMS, time_column='datetime', min_temp_column='min_temp', max_temp_column='max_temp')

In [4]:
#Using year 2024 start date and end date for mine gdd
start_date = pd.to_datetime("5-11-2023").date()
end_date = pd.to_datetime("11-8-2023").date()

AGDD for StPaul corn 2023

In [5]:
corn_gdd_StPaulGEMS = StPaul_GEMS_2023.con_gdd(start_date, end_date)
print(corn_gdd_StPaulGEMS)

     clouds   datetime  dewpt  dhi  dni  ghi  max_dhi  max_dni  max_ghi  \
130      88 2023-05-11   13.9   55  442  331      120      919      933   
131      86 2023-05-12   13.8   55  444  332      120      919      935   
132     100 2023-05-13   11.6   55  445  333      120      919      936   
133      85 2023-05-14    8.4   55  446  335      121      920      938   
134      42 2023-05-15    6.6   56  448  336      121      920      940   
..      ...        ...    ...  ...  ...  ...      ...      ...      ...   
307      46 2023-11-04   -1.5   29  250  118       94      793      478   
308      75 2023-11-05    3.5   28  238  112       94      791      473   
309      82 2023-11-06    4.4   28  247  115       94      789      468   
310      99 2023-11-07    1.0   28  245  113       93      787      463   
311      84 2023-11-08    2.2   28  244  112       93      784      458   

     max_temp  ...  t_solar_rad  temp          ts  wind_dir  wind_gust_spd  \
130     75.92  ...   